Standortkarten Print Skript

In [17]:
import sys
import os
import psycopg2
import configparser
import requests
import json
import subprocess
from requests.auth import HTTPBasicAuth

In [25]:
def process_exists(process_name):
    call = 'TASKLIST', '/FI', 'imagename eq %s' % process_name
    # use buildin check_output right away
    output = subprocess.check_output(call)
    # check in last line for process name
    last_line = output.strip().split('\r\n')[-1]
    # because Fail message could be translated
    return last_line.lower().startswith(process_name.lower())

In [11]:
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding("utf-8")
sys.stdout = stdout
#disable proxy for localhost
os.environ['NO_PROXY'] = '127.0.0.1'
config=configparser.ConfigParser()

In [26]:
try:
    config.read("printconfig.ini")
except:
    print "exception while reading config file"

In [28]:
if not process_exists("postgres.exe"):
    print "Datenbank ist nicht offen!"
    
    #Datenbank öffnen
    #ggf. erst mal Datenbank importieren
    #https://stackoverflow.com/questions/15107714/wait-process-until-all-subprocess-finish
    #am Ende natürlich DB schließen
else:
    print "Datenbank offen!"

Datenbank ist nicht offen!


In [13]:
r=requests.post(config.get("webservice","url"),
                auth=HTTPBasicAuth(config.get("webservice","username"),config.get("webservice","password")),
                data={"command":"fetchAllPrintingQueue"},
               )
if r.status_code == 200:
    queue=json.loads(r.text)
    for job in queue:
        print job
        #start db and wait some seconds
        
        #import mapnik
        sys.path.append(config.get("mapnik","pythonPath")) 
        import mapnik
        
else:
    print "konnte Printqueue nicht holen",r.status_code,r.text


{u'bundesland': 1, u'branche': 1, u'updated': 0, u'queued': 1, u'dataset': u'_v2'}
{u'bundesland': 1, u'branche': 4, u'updated': 0, u'queued': 1, u'dataset': u'_v2'}
{u'bundesland': 1, u'branche': 2, u'updated': 0, u'queued': 1, u'dataset': u'_v2'}
{u'bundesland': 1, u'branche': 3, u'updated': 0, u'queued': 1, u'dataset': u'_v2'}
{u'bundesland': 1, u'branche': 5, u'updated': 0, u'queued': 1, u'dataset': u'_v2'}
{u'bundesland': 1, u'branche': 6, u'updated': 0, u'queued': 1, u'dataset': u'_v2'}
{u'bundesland': 2, u'branche': 6, u'updated': 0, u'queued': 1, u'dataset': u'_v2'}
{u'bundesland': 1, u'branche': 7, u'updated': 0, u'queued': 1, u'dataset': u'_v2'}
{u'bundesland': 1, u'branche': 8, u'updated': 0, u'queued': 1, u'dataset': u'_v2'}
{u'bundesland': 1, u'branche': 9, u'updated': 0, u'queued': 1, u'dataset': u'_v2'}
